In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95!important; }</style>"))

In [2]:
# Instalando bibliotecas
!pip install scrapy

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [74]:
import requests
from scrapy.http import TextResponse

In [75]:
def send_request_to_url(site_url):
    # py_res --> Response nativa do Python
    py_res = requests.get(site_url)
    #print(py_res)

    # scrapy_res --> Response própria para ser utilizada no scrapy
    scrapy_res = TextResponse(py_res.url, body=py_res.text, encoding='utf-8')
    #print(scrapy_res)
    
    return scrapy_res

In [76]:
url = 'https://rebep.org.br/revista/article/view/'

In [65]:
#scrapy_res = send_request_to_url(url)

In [66]:
## volume:
#scrapy_res.xpath('//div[@class="page page_article"]//ol//li[3]//a/text()').extract()

In [67]:
#tipo de publicação:
#scrapy_res.xpath('//div[@class="page page_article"]//ol//li[last()]/text()').extract()

In [68]:
#título:
#scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//h1[@class="page_title"]/text()').extract()

In [69]:
#autor:
#scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//ul[@class="item authors"]//span[@class="name"]/text()').extract()

In [70]:
#resumo - opção 1:
#scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//div[@class="item abstract"]//p/text()').extract()

In [71]:
#resumo - opção 2
#scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//div[@class="item abstract"]/text()').extract()

In [77]:
import time
# Navega pelo website coletando as publicações da RBEP
# texts é uma lista que irá salvar dicionários
# cada dicionário é uma publicação da RBEP
texts = []

# iteramos sobre as paginas - verificação manual do número de páginas
for i in range(2,1544):
    # observacao: estamos fazendo requests em um loop. Adicionaremos um delay
    scrapy_res = send_request_to_url(url + str(i))
    
    # delay de 1s
    time.sleep(1)
    
    # dicionário
    text_dict = {}
    
    # link da publicação
    text_dict['link'] = url + str(i)
    
    # edição:
    text_dict['edicao'] = scrapy_res.xpath('//div[@class="page page_article"]//ol//li[3]//a/text()').extract()
    
    # tipo de publicação:
    text_dict['tipo_trabalho'] = scrapy_res.xpath('//div[@class="page page_article"]//ol//li[last()]/text()').extract()

    # título:
    text_dict['titulo'] = scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//h1[@class="page_title"]/text()').extract()

    # autor:
    text_dict['autor'] = scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//ul[@class="item authors"]//span[@class="name"]/text()').extract()

    # resumo:
    resumo_opcao_1 = scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//div[@class="item abstract"]//p/text()').extract()
    if resumo_opcao_1 != []:
        text_dict['resumo'] = resumo_opcao_1
    else:
        resumo_opcao_2 = scrapy_res.xpath('//div[@class="page page_article"]//article[@class="obj_article_details"]//div[@class="row"]//div[@class="item abstract"]/text()').extract()
        if len(resumo_opcao_2) > 0:
            # inspecionando os casos, o resumo se encontra na posição 2
            text_dict['resumo'] = resumo_opcao_2[1]
        else:
            text_dict['resumo'] = []
    
    texts.append(text_dict)

In [78]:
texts

[{'autor': ['\n\t\t\t\t\t\t\t\tNeide Lopes Patarra\n\t\t\t\t\t\t\t'],
  'edicao': ['\n\t\t\t\tv. 1 n. 1/2 (1984)\n\t\t\t'],
  'link': 'https://rebep.org.br/revista/article/view/2',
  'resumo': [],
  'tipo_trabalho': ['\n\t\t\t\t\t\t\t\t\tEditorial\n\t\t\t\t\t\t'],
  'titulo': ['\n\t\tApresentação\n\t']},
 {'autor': ['\n\t\t\t\t\t\t\t\tDonald R. Saywer\n\t\t\t\t\t\t\t'],
  'edicao': ['\n\t\t\t\tv. 1 n. 1/2 (1984)\n\t\t\t'],
  'link': 'https://rebep.org.br/revista/article/view/3',
  'resumo': '\n\t\t\t\t\tSão examinados historicamente os processos de avanço e recuo espacial das atividades econômicas e da população, procurando-se ligações a nível estrutural ente os movimentos centrífugos da fronteira e os movimentos centrípetos do desenvolvimento capitalista. No período urbano-industrial, enquanto as atividades econômicas e a população concentravam-se em polos urbanos, parte da população excedente dirigiu-se a novas fronteiras no interior, em função da ampliação do âmbito da circulação de

In [79]:
import pandas as pd

In [80]:
# problema: transformar um dicionario com itens de tamanhos diferentes em um data frame
# solução
# https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths
# "problema", que é resolvível, como os dicionários tem tamanhos diferentes,
# ao criar o data frame apenas a primeira linha tem conteúdo, as demais ficam com NaN

texts_df = []
for i in range(0,len(texts)):
    df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in texts[i].items() ]))
    texts_df.append(df)

In [81]:
pd.concat(texts_df,ignore_index=True).to_csv('trabalhos_rbep.csv')